In [1]:
!pip install torch


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install transformers pandas


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

/Users/kushagrawadhwa/Desktop/PDF parser/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/kushagrawadhwa/Desktop/PDF parser/.venv/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/kushagrawadhwa/Desktop/PDF parser/.venv/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/kushagrawadhwa/Desktop/PDF parser/.venv/lib/python3.9/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and 

In [4]:
!pip install SentencePiece


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install spacy

NATIVE QUESTIONS

In [11]:
import json
import re
import os
import pandas as pd

def save_to_csv(qa_pairs, csv_file):
    df = pd.DataFrame(qa_pairs, columns=['question', 'response'])
    if os.path.exists(csv_file):
        df.to_csv(csv_file, mode='a', index=False, header=False)
        print(f"Q&A pairs have been appended to {csv_file}")
    else:
        df.to_csv(csv_file, index=False)
        print(f"New CSV file {csv_file} has been created with Q&A pairs")

def save_to_text(qa_pairs, text_file):
    with open(text_file, 'a', encoding='utf-8') as file:
        for question, answer in qa_pairs:
            file.write(f"Question: {question}\n")
            file.write(f"Answer: {answer}\n\n")
    print(f"Q&A pairs have been saved to {text_file}")

def generate_qa(json_file, document_name):
    if not os.path.exists(json_file) or os.path.getsize(json_file) == 0:
        raise ValueError(f"The file {json_file} does not exist or is empty.")
    
    with open(json_file, 'r') as file:
        try:
            data = json.load(file)
        except json.JSONDecodeError as e:
            raise ValueError(f"Error decoding JSON from file {json_file}: {e}")
    
    questions_answers = []
      
    for key, value in data.items():
        question = re.sub(r'^\d+(\.\d+)*\.\s*', '', key).strip()
        question = f"What is the {question} in {document_name}?"
        answer = re.sub(r'\\u[0-9a-fA-F]{4}', "", value)
        answer = re.sub(r'^[^a-zA-Z0-9\s]+', '', answer, flags=re.MULTILINE)
            
        questions_answers.append((question, answer))
    
    return questions_answers

json_file = '/Users/kushagrawadhwa/Desktop/PDF parser/llama_parser/output_policy2.json'
document_name = 'the Handbook'
csv_file = 'native_qa/native_qa_pairs2.0.csv'
text_file = 'native_qa/native_qa_combined.txt'

try:
    qa_pairs = generate_qa(json_file, document_name)
    # save_to_csv(qa_pairs, csv_file)
    save_to_text(qa_pairs, text_file)
except ValueError as e:
    print(e)    

Q&A pairs have been saved to native_qa/native_qa_combined.txt


T-5 QUESTIONS

In [2]:
import json
import os
import pandas as pd
from transformers import AutoModelWithLMHead, AutoTokenizer

t5_model_name = "mrm8488/t5-base-finetuned-question-generation-ap"
t5_tokenizer = AutoTokenizer.from_pretrained(t5_model_name)
model = AutoModelWithLMHead.from_pretrained(t5_model_name)

def generate_question(answer, context):
    input_text = f"answer: {answer} context: {context}"
    input_ids = t5_tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(input_ids, max_length=64)
    question = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    if question.startswith("question: "):
        question = question[len("question: "):]
    return question


def process_json_file(file_path, document_name):
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    qa_pairs = []
    for heading, content in data.items():
        context = str(content)
        answer = heading  
        generated_question = generate_question(answer, context) 
        qa_pairs.append((generated_question, context))
    return qa_pairs

def save_to_csv(qa_pairs, csv_file):
    df = pd.DataFrame(qa_pairs, columns=['question', 'response'])
    if os.path.exists(csv_file):
        df.to_csv(csv_file, mode='a', index=False, header=False)
        print(f"Q&A pairs have been appended to {csv_file}")
    else:
        df.to_csv(csv_file, index=False)
        print(f"New CSV file {csv_file} has been created with Q&A pairs")

def save_to_text(qa_pairs, text_file):
    with open(text_file, 'a', encoding='utf-8') as file:
        for question, answer in qa_pairs:
            file.write(f"Question: {question}\n")
            file.write(f"Answer: {answer}\n\n")
    print(f"Q&A pairs have been saved to {text_file}")
    
file_path = '/Users/kushagrawadhwa/Desktop/PDF parser/llama_parser/output_handbook5_wo_inst.json'
document_name = 'the Internal Mobility Policy'
csv_file = 't-5_qa/t-5_qa_pairs.csv'
text_file = 't-5_qa/t-5_qa_combined.txt'


try:
    qa_pairs = process_json_file(file_path, document_name)
    # save_to_csv(qa_pairs, csv_file)
    save_to_text(qa_pairs, text_file)
except ValueError as e:
    print(e)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/Users/kushagrawadhwa/Desktop/PDF parser/.venv/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1564: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Token 

Q&A pairs have been saved to t-5_qa/t-5_qa_combined.txt


In [1]:
import json
import os
import pandas as pd
from transformers import AutoModelWithLMHead, AutoTokenizer

t5_model_name = "mrm8488/t5-base-finetuned-question-generation-ap"
t5_tokenizer = AutoTokenizer.from_pretrained(t5_model_name)
model = AutoModelWithLMHead.from_pretrained(t5_model_name)

text = 'Construction of the mausoleum was completed in 1648, but work continued on other phases of the project for another five years. The first ceremony held at the mausoleum was an observance by Shah Jahan, on 6 February 1643, of the 12th anniversary of the death of Mumtaz Mahal. The Taj Mahal complex is believed to have been completed in its entirety in 1653 at a cost estimated at the time to be around ₹5 million, which in 2023 would be approximately ₹35 billion (US$77.8 million).'

def generate_question(text):
    input_text = f"Text: {text}"
    input_ids = t5_tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(input_ids, max_length=64)
    question = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    if question.startswith("question: "):
        question = question[len("question: "):]
        print(question)
    # return question

try:
    generate_question(text)
except ValueError as e:
    print(e)

/Users/kushagrawadhwa/Desktop/PDF parser/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/kushagrawadhwa/Desktop/PDF parser/.venv/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/kushagrawadhwa/Desktop/PDF parser/.venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remov

What was the estimated cost of the Taj Mahal complex in 1653?


BERT QUESTIONS

In [ ]:
import json
import os
import pandas as pd
import torch
from transformers import BertForQuestionAnswering, BertTokenizer
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

model_name = "distilbert/distilbert-base-uncased"
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

def generate_question(context, answer):
    inputs = tokenizer.encode_plus(answer, context, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    outputs = model(**inputs)
    start_scores, end_scores = outputs.start_logits, outputs.end_logits
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)
    question = tokenizer.decode(input_ids[start_index:end_index+1])

    return question

def process_json_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    qa_pairs = []
    for heading, content in data.items():
        context = str(content)
        answer = heading  
        generated_question = generate_question(context, answer)
        qa_pairs.append((generated_question, context))
    return qa_pairs

def save_to_csv(qa_pairs, csv_file):
    df = pd.DataFrame(qa_pairs, columns=['question', 'response'])
    if os.path.exists(csv_file):
        df.to_csv(csv_file, mode='a', index=False, header=False)
        print(f"Q&A pairs have been appended to {csv_file}")
    else:
        df.to_csv(csv_file, index=False)
        print(f"New CSV file {csv_file} has been created with Q&A pairs")

file_path = '/Users/kushagrawadhwa/Desktop/PDF parser/llama_parser/output_policy1.json'
csv_file = 'bert_qa/bert_qa_pairs.csv'

try:
    qa_pairs = process_json_file(file_path)
    save_to_csv(qa_pairs, csv_file)
except ValueError as e:
    print(e)

In [41]:
#Preprocess csv files
import pandas as pd
df=pd.read_csv("/Users/kushagrawadhwa/Desktop/PDF parser/Q|A Pairs/native_qa/native_qa_pairs2.0.csv")

In [42]:
df

,question,response
0,What is the Internal Mobility Guidelines in th...,NaN
1,What is the 'Live Your Potential' - Internal M...,NaN
2,What is the Purpose in the Handboook?,The purpose of these guidelines is to ensure t...
3,What is the Scope in the Handboook?,These guidelines apply to all full-time employ...
4,What is the Employee Eligibility in the Handbo...,In order to be eligible for a movement to anot...
...,...,...
888,What is the Workplace Behavior in the Handboook?,Employees are expected to maintain a positive ...
889,What is the Confidentiality in the Handboook?,All employees must maintain the confidentialit...
890,What is the Attendance and Punctuality in the ...,Employees are expected to arrive on time for t...
891,What is the Communication in the Handboook?,Effective communication is key to the success ...


In [43]:
df.isnull().sum()

question      0
response    130
dtype: int64

In [44]:
df_new=df.dropna()

In [45]:
df_new=df_new.drop_duplicates()

In [46]:
df_new.to_csv('native_qa/native_qa_pairs2.0_final.csv', index=False)

In [47]:
df_new

,question,response
2,What is the Purpose in the Handboook?,The purpose of these guidelines is to ensure t...
3,What is the Scope in the Handboook?,These guidelines apply to all full-time employ...
4,What is the Employee Eligibility in the Handbo...,In order to be eligible for a movement to anot...
5,What is the Process & Approvals in the Handboook?,An email announcing the available open positio...
7,What is the Tech Stack in the Handboook?,Details about the technology stack used within...
...,...,...
888,What is the Workplace Behavior in the Handboook?,Employees are expected to maintain a positive ...
889,What is the Confidentiality in the Handboook?,All employees must maintain the confidentialit...
890,What is the Attendance and Punctuality in the ...,Employees are expected to arrive on time for t...
891,What is the Communication in the Handboook?,Effective communication is key to the success ...


In [ ]:
import pandas as pd